<a href="https://colab.research.google.com/github/d33pd/d33pd/blob/main/sdreddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import praw
import openai
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
import asyncio
import aiohttp
import time
import psycopg2
from psycopg2 import pool
import logging
import nltk
import atexit

nltk.download('vader_lexicon')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
SENTIMENT_THRESHOLD = 0.5
MAX_TOKENS = 2048
DB_POOL_MIN_CONN = 1
DB_POOL_MAX_CONN = 10

# Set up Reddit API credentials
reddit = praw.Reddit(
    client_id="your_client_id",
    client_secret="your_client_secret",
    user_agent="your_user_agent",
    username="your_username",
    password="your_password",
)

# Set up OpenAI API credentials
openai.api_key = "your_openai_api_key"

# Load the Spacy model
nlp = spacy.load("en_core_web_sm")

# Set up the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Create a database connection pool
db_pool = psycopg2.pool.SimpleConnectionPool(DB_POOL_MIN_CONN, DB_POOL_MAX_CONN,
                                             user='your_db_user',
                                             password='your_db_password',
                                             host='localhost',
                                             port='5432',
                                             database='your_database_name')

# Create a table to store comments if it doesn't exist
def init_db():
    conn = db_pool.getconn()
    with conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS comments (
                    id TEXT PRIMARY KEY,
                    body TEXT,
                    sentiment REAL
                )
            """)
    db_pool.putconn(conn)

init_db()  # Initialize the database

async def process_comment(comment):
    """Processes and stores comments in the database."""
    sentiment = analyze_sentiment(comment.body)
    conn = db_pool.getconn()
    try:
        with conn.cursor() as cursor:
            cursor.execute("INSERT INTO comments (id, body, sentiment) VALUES (%s, %s, %s) ON CONFLICT (id) DO NOTHING",
                           (comment.id, comment.body, sentiment))
        conn.commit()
    except psycopg2.Error as e:
        logging.error(f"Database error: {e}")
    finally:
        db_pool.putconn(conn)

async def generate_response(session, prompt):
    """Generates a response to a prompt using OpenAI API."""
    retries = 3
    for attempt in range(retries):
        try:
            async with session.post('https://api.openai.com/v1/completions', json={
                "model": "text-davinci-002",
                "prompt": prompt,
                "max_tokens": MAX_TOKENS,
                "temperature": 0.7,
            }, headers={"Authorization": f"Bearer {openai.api_key}"}) as response:
                response_data = await response.json()
                return response_data['choices'][0]['text'].strip()
        except Exception as e:
            logging.error(f"Error with OpenAI API (attempt {attempt + 1}): {e}")
            if attempt < retries - 1:
                await asyncio.sleep(2 ** attempt)  # Exponential backoff
    return None

def analyze_sentiment(comment_body):
    """Analyzes the sentiment of a comment."""
    sentiment = sia.polarity_scores(comment_body)
    return sentiment['compound']

async def monitor_subreddit(session, subreddit_name):
    """Monitors a subreddit for new comments and replies to them."""
    subreddit = reddit.subreddit(subreddit_name)
    keywords = {"what", "when", "why", "how", "who", "where", "which", "help", "advice"}

    async for comment in subreddit.stream.comments(skip_existing=True):
        if any(word in comment.body.lower() for word in keywords) or "?" in comment.body.lower():
            try:
                await process_comment(comment)
                response = await generate_response(session, f"Respond helpfully to this Reddit comment:\n{comment.body}")
                if response:
                    comment.reply(response)
                    logging.info(f"Replied to comment: {comment.body}")
            except Exception as e:
                logging.error(f"Error replying to comment: {e}")

async def like_and_repost(session, subreddit_name, num_comments):
    """Likes and reposts valuable comments based on sentiment analysis."""
    subreddit = reddit.subreddit(subreddit_name)
    comments = subreddit.comments(limit=num_comments)

    for comment in comments:
        try:
            sentiment = analyze_sentiment(comment.body)
            if sentiment > SENTIMENT_THRESHOLD:
                comment.upvote()
                logging.info(f"Liked comment: {comment.body}")

                # Repost the comment
                title = f"Repost: {comment.body[:50]}..."
                subreddit.submit(title=title, selftext=comment.body)
                logging.info(f"Reposted comment: {title}")
        except praw.exceptions.APIException as e:
            logging.warning(f"API error: {e}")
            await asyncio.sleep(10)  # Handle rate limits gracefully
        except Exception as e:
            logging.error(f"Unexpected error: {e}")

async def main():
    async with aiohttp.ClientSession() as session:
        await asyncio.gather(
            monitor_subreddit(session, "your_subreddit_name"),
            like_and_repost(session, 'your_subreddit_name', 500),
        )

# Register exit function to close the database pool
atexit.register(lambda: db_pool.closeall())

# Run the main function
if __name__ == "__main__":
    asyncio.run(main())
